In [3]:
import tensorflow as tf
from keras.datasets import cifar10

In [46]:
from keras.utils import to_categorical
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.transpose(x_train, (0,2,3,1))
x_test = np.transpose(x_test, (0,2,3,1))
# x_train = x_train[:10000]
# x_test = x_test[:100]
# y_train = y_train[:10000]
# y_test = y_test[:100]

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(x_train.shape)
print(y_train.shape)
print(y[0])

(50000, 3, 32, 32)
(50000, 1)
(50000, 32, 32, 3)
(50000, 10)
Tensor("strided_slice:0", shape=(10,), dtype=float32)


In [22]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pooling_2X2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W)+b)

def fully_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

In [24]:
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

conv_1 = conv_layer(X, [5,5,3,32])
conv1_pool = max_pooling_2X2(conv_1)

conv_2 = conv_layer(conv1_pool, [5,5,32,64])
conv2_pool = max_pooling_2X2(conv_2)
conv2_flat = tf.reshape(conv2_pool, [-1,8*8*64])

full_1 = tf.nn.relu(fully_layer(conv2_flat, 1024))
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

full_2 = fully_layer(full1_drop, 10)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=full_2)
gd = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(full_2, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [25]:
STEPS = 10
MINIBATCH_SIZE = 100
init = tf.global_variables_initializer()

split_X = np.split(x_train, int(x_train.shape[0]/MINIBATCH_SIZE))
split_y = np.split(y_train, int(y_train.shape[0]/MINIBATCH_SIZE))

n_test = int(x_test.shape[0]/MINIBATCH_SIZE)
split_X2 = np.split(x_test, int(x_test.shape[0]/MINIBATCH_SIZE))
split_y2 = np.split(y_test, int(y_test.shape[0]/MINIBATCH_SIZE))

with tf.Session() as sess:
    sess.run(init)    
    for i in range(STEPS):
        k = 0
        if i % 10 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={X:split_X[k], y:split_y[k], keep_prob:1.0})
            print("step {}, training accuracy {}".format(i, train_accuracy))
        sess.run(gd, feed_dict={X:split_X[k], y:split_y[k], keep_prob:0.5})
        k += 1
    test_accuracy = np.mean([sess.run(accuracy, feed_dict={X:split_X2[i], y:split_y2[i], keep_prob:1.0}) for i in range(n_test)])
    print("test accuracy: {:.4}%".format(test_accuracy * 100))

step 0, training accuracy 0.10999999940395355
test accuracy: 15.0%


In [48]:
g2 = tf.Graph()

STEPS = 100
MINIBATCH_SIZE = 100

split_X = np.split(x_train, int(x_train.shape[0]/MINIBATCH_SIZE))
split_y = np.split(y_train, int(y_train.shape[0]/MINIBATCH_SIZE))

n_test = int(x_test.shape[0]/MINIBATCH_SIZE)
split_X2 = np.split(x_test, int(x_test.shape[0]/MINIBATCH_SIZE))
split_y2 = np.split(y_test, int(y_test.shape[0]/MINIBATCH_SIZE))

with g2.as_default():
    X = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y = tf.placeholder(tf.float32, [None, 10])
    keep_prob = tf.placeholder(tf.float32)

    C1, C2, C3 = 30, 50, 80 
    F1 = 500
    
    conv1_1 = conv_layer(X, [3,3,3,C1])
    conv1_2 = conv_layer(conv1_1, [3,3,C1,C1])
    conv1_3 = conv_layer(conv1_2, [3,3,C1,C1])
    conv1_pool = max_pooling_2X2(conv1_3)
    conv1_drop = tf.nn.dropout(conv1_pool, keep_prob=keep_prob)

    conv2_1 = conv_layer(conv1_drop, [3,3,C1,C2])
    conv2_2 = conv_layer(conv2_1, [3,3,C2,C2])
    conv2_3 = conv_layer(conv2_2, [3,3,C2,C2])
    conv2_pool = max_pooling_2X2(conv2_3)
    conv2_drop = tf.nn.dropout(conv2_pool, keep_prob=keep_prob)

    conv3_1 = conv_layer(conv2_drop, shape=[3, 3, C2, C3]) 
    conv3_2 = conv_layer(conv3_1, shape=[3, 3, C3, C3]) 
    conv3_3 = conv_layer(conv3_2, shape=[3, 3, C3, C3]) 
    conv3_pool = tf.nn.max_pool(conv3_3, ksize=[1, 8, 8, 1], strides=[1, 8, 8, 1],padding='SAME') 
    conv3_flat = tf.reshape(conv3_pool, [-1, C3])

    
    conv3_drop = tf.nn.dropout(conv3_flat, keep_prob=keep_prob)
    full1 = tf.nn.relu(fully_layer(conv3_flat, F1)) 
    full1_drop = tf.nn.dropout(full1, keep_prob=keep_prob)
    full_2 = fully_layer(full1_drop, 10)

    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=full_2)
    gd = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    correct_prediction = tf.equal(tf.argmax(full_2, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)    
        for i in range(STEPS):
            for k in range(MINIBATCH_SIZE):
                if i % 10 == 0:
                    train_accuracy = sess.run(accuracy, feed_dict={X:split_X[k], y:split_y[k], keep_prob:1.0})
                    print("step {}, training accuracy {}".format(i, train_accuracy))
                sess.run(gd, feed_dict={X:split_X[k], y:split_y[k], keep_prob:0.5})
        test_accuracy = np.mean([sess.run(accuracy, feed_dict={X:split_X2[i], y:split_y2[i], keep_prob:1.0}) for i in range(n_test)])
        print("test accuracy: {:.4}%".format(test_accuracy * 100))

step 0, training accuracy 0.12999999523162842
step 0, training accuracy 0.09000000357627869
step 0, training accuracy 0.05000000074505806
step 0, training accuracy 0.09000000357627869
step 0, training accuracy 0.10000000149011612
step 0, training accuracy 0.14000000059604645
step 0, training accuracy 0.07000000029802322
step 0, training accuracy 0.10000000149011612
step 0, training accuracy 0.14000000059604645
step 0, training accuracy 0.11999999731779099
step 0, training accuracy 0.1899999976158142
step 0, training accuracy 0.11999999731779099
step 0, training accuracy 0.11999999731779099
step 0, training accuracy 0.11999999731779099
step 0, training accuracy 0.10999999940395355
step 0, training accuracy 0.05000000074505806
step 0, training accuracy 0.10999999940395355
step 0, training accuracy 0.11999999731779099
step 0, training accuracy 0.07999999821186066
step 0, training accuracy 0.09000000357627869
step 0, training accuracy 0.12999999523162842
step 0, training accuracy 0.0500000

KeyboardInterrupt: 